## This notebook calls 30 days of transactions for NFT named collections on OpenSea - an NFT marketplace.

## Note: Running the notebook requires an API key from OpenSea.

In [1]:
# imports
import pandas as pd
import datetime

import requests

import os
from dotenv import load_dotenv

In [2]:
# load API key
load_dotenv()
opensea_api_key = os.getenv("OPENSEA_API_KEY")

In [3]:
# input collection name (e.g., "nakamigos")
collection = input("Which NFT collection on OpenSea do you want to call?")

In [4]:
# set today's date
today = datetime.datetime.now()

# set number of days in past
past_date = today - datetime.timedelta(days=30)

# convert to unix (starting 1970!)
unix_past_date = int(past_date.timestamp())

# create blank df to store page data
events_df = pd.DataFrame()

# initialize next_event variable
next_event = None

# PAGINATION
while True:
    url = f"https://api.opensea.io/api/v2/events/collection/{collection}?after={unix_past_date}&event_type=sale"
    if next_event:
        # This adds the next event value (and required syntax) to the URL string
        url += f"&next={next_event}"

    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # formatting as json
    events_json = response.json()

    # get the "asset events"
    events_data = events_json.get("asset_events", [])
    
    # store to df
    page_events_df = pd.json_normalize(events_data)
    
    # concatenate to master df
    events_df = pd.concat([events_df, page_events_df], ignore_index=True)
    
    # update next_event for pagination
    next_event = events_json.get("next")

    # if there are no more pages, exit the loop
    if not next_event:
        break

# preview
events_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.display_animation_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,0xe798a7e92cf55e474f49249e186fbb531a7c724f0fad...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1725740747,1,0x1f2874ec67c8a2fd64994b64c0a3469b499140e3,0xf74f8ad40b17887b0379d87c55c063dc2861aa2f,0x1c975fd3d1f3f4f88634b03bacbceb580bb861e39f72...,1725740747,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-09-07T20:25:50.410519,False,False,150000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1,sale,0xf135a5236eb22b365582c45e0fe9f2555f867149f52e...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1725740747,1,0xda48a4073e29206657ef7759dbfb3c6ce15d3af9,0xf74f8ad40b17887b0379d87c55c063dc2861aa2f,0x1c975fd3d1f3f4f88634b03bacbceb580bb861e39f72...,1725740747,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-09-07T20:25:50.253407,False,False,168000000000000000,0x0000000000000000000000000000000000000000,18,ETH
2,sale,0x95b590549debfce7484f804b2932fab5464b109f123e...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1725740747,1,0x175df70833652ebe593f224a97879f3b4d07e8f4,0xf74f8ad40b17887b0379d87c55c063dc2861aa2f,0x1c975fd3d1f3f4f88634b03bacbceb580bb861e39f72...,1725740747,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-09-07T20:25:50.081941,False,False,144000000000000000,0x0000000000000000000000000000000000000000,18,ETH
3,sale,0x642304c2a7b61f00a1094adb8b072076b158b51f0bf5...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1725738647,1,0xc8a738228bbe4726250bc518819d6879ce467968,0x9f8f59ee9c6450dedb68990fb7a02788f003af9d,0xf52f0d2c2fbf06d0932ab35e248a8a043e04730c1cb7...,1725738647,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-09-07T19:50:50.640070,False,False,132500000000000000,0x0000000000000000000000000000000000000000,18,ETH
4,sale,,ethereum,,1725728771,1,0x8b49899d2e0285c6d7b9523037866f79e35da88c,0xe580a803a3b4b18bbb1aa8d6d7a55a640e8ec787,0x66432019a26a918878473b4167fc4524b120195af307...,1725728771,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-09-07T17:06:14.106290,False,False,123900000000000000,0x0000000000000000000000000000000000000000,18,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,sale,0xee840e65ec811989ff341b50f8e77a3e0a3488dde44a...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1723210235,1,0xe5cb840fd8f158058972a14938fd49472bd40634,0x9c0f056729a31f323808d9a61bd7246170c1185d,0x4289e2431aeed035799b1f20c496a63342c0b4e6ed9f...,1723210235,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-08-15T00:48:13.905978,False,False,144700000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1168,sale,0x6ec635962805909de625af19ac6a65e5a06fd9e6c09a...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1723207091,1,0x4c0b178fa747e4b7e2d6819677c37072c101937c,0xa7df793d7957a2a1f9a5461619ead8793187d6d3,0xfc68bc64b899d2217b16874fdf47dfeb22a7e8a57817...,1723207091,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-08-09T13:37:25.549887,False,False,78000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1169,sale,,ethereum,,1723195211,1,0x1e64c293de2ecd58a4ea48a511cd1bed386ac15a,0xb2d0b117d36cd9a74bafdb3d01ff754b30a66e23,0xd83941e6927489b7ec6017e761f4012212a5ca968fc7...,1723195211,...,None,https://ipfs.io/ipfs/QmaN1jRPtmzeqhp6s3mR1SRK4...,https://opensea.io/assets/ethereum/0xd774557b6...,2024-08-10T09:42:13

In [5]:
# create CSV name
csv_name = f"{collection}_events.csv"

# write df to csv
events_df.to_csv(csv_name)